In [1]:
from __future__ import print_function
from __future__ import absolute_import
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('white')

Visualize the sjtuvis dataset!

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
from textwrap import wrap

from pysaliency.external_datasets.sjtuvis import TextDescriptor

data_location = "../../datasets/test"
text_descriptor = TextDescriptor('../../datasets/test/original_sjtuvis_dataset/text.xlsx')

def load_images_with_versions(folder, versions):
    images = {}
    for filename in os.listdir(folder):
        basename, ext = os.path.splitext(filename)
        key_basename = basename
        for version in versions:
            key_basename = key_basename.replace(f'_{version}', '')
        if key_basename not in images:
            images[key_basename] = {}

        for version in versions:
            if basename.endswith(f'_{version}') or (version == 'overall' and '_' not in basename):
                img = Image.open(os.path.join(folder, filename))
                if img is not None:
                    images[key_basename][version] = img
    return images

def create_image_grid_with_captions(ori_images, map_images, get_caption, rows=8, cols=10):
    fig = plt.figure(figsize=(40, 30))  # Adjusted figure size
    gs = gridspec.GridSpec(rows, cols, hspace=0.4, wspace=0.0)  # Reduced hspace

    for i, (basename, versions) in enumerate(ori_images.items()):
        row = i // (cols // 5) 
        col = (i % (cols // 5)) * 5

        ax = plt.subplot(gs[row, col:col+1])
        ori_image = ori_images[basename]['0'].resize((1280, 960))
        ax.imshow(ori_image)
        ax.axis('off')
        
        for j, (version, img) in enumerate(versions.items()):
            ax = plt.subplot(gs[row, col + j + 1:col + j + 2])
            map_image = map_images[basename][version].resize((1280, 960))
            ax.imshow(map_image)
            text_description = text_descriptor.get_description(f'{basename}_{version}.png')
            title = ax.set_title("\n".join(wrap(text_description, 35)), fontsize=15)
            title.set_y(1.05)
            title.set_multialignment('center')
            title.set_linespacing(1.5)
            ax.axis('off')

        if i >= rows * cols // 5 - 1:
            break

    plt.subplots_adjust(top=0.99, bottom=0.01, left=0.01, right=0.99, hspace=0.01, wspace=0.01)
    plt.show()


# Dummy function for generating captions
def get_caption(version, basename):
    return f"Caption for version {version} of {basename}"

# Load your images
# Replace 'path_to_images' with the actual path to your images

ori_images = load_images_with_versions('../../datasets/test/original_sjtuvis_dataset/image', versions=['overall', '0', '1', '2', '3'])
map_images = load_images_with_versions('../../datasets/test/original_sjtuvis_dataset/map', versions=['overall', '0', '1', '2', '3'])

ori_images = {k: v for k, v in ori_images.items() if len(v.keys()) == 4}
map_images = {k: v for k, v in map_images.items() if len(v.keys()) == 4}
# print(ori_images)
# print(map_images)

create_image_grid_with_captions(ori_images,map_images, get_caption)

Visualize the model from stimuli.hdf5 and fixation.hdf5 !

In [ ]:
import sys
import pysaliency
from pysaliency.external_datasets.sjtuvis import TextDescriptor
import matplotlib.pyplot as plt
import pysaliency

mit_stimuli, mit_fixations = pysaliency.external_datasets.get_mit1003_onesize(location="../../datasets/")
mit_stimuli, mit_fixations = pysaliency.external_datasets.get_DUT_OMRON(location="../../datasets/")
# mit_stimuli, mit_fixations = pysaliency.external_datasets.get_FIGRIM(location="../../datasets/")
# mit_stimuli, mit_fixations = pysaliency.external_datasets.get_NUSEF_public(location="../../datasets/")
# mit_stimuli, mit_fixations = pysaliency.external_datasets.get_OSIE(location="../../datasets/")
# stimuli_train, stimuli_val, stimuli_test, fixations_train, fixations_val = pysaliency.external_datasets.get_SALICON(location="../../datasets/")
# mit_stimuli, mit_fixations = stimuli_train, fixations_train
mit_stimuli, mit_fixations = pysaliency.external_datasets.get_PASCAL_S(location="../../datasets/")
mit_stimuli, mit_fixations = pysaliency.external_datasets.get_toronto(location="../../datasets/")

import os
from PIL import Image
import matplotlib.pyplot as plt

def create_image_grid(mit_stimuli, mit_fixations, rows=18, cols=6):
    
    fig = plt.figure(figsize=(20, 85))

    i = 0
    cut_off = rows * cols // 2
    for i, stimulus in enumerate(mit_stimuli[:cut_off]):
        fixation = mit_fixations[mit_fixations.n == i] 
             
        # img = Image.fromarray(stimulus.stimulus_data)
        # img = img.resize((640, 480))
        img = stimulus.stimulus_data
        row, col = divmod(i, cols // 2)

        ax1 = fig.add_subplot(rows, cols, row * cols + col * 2 + 1)
        ax2 = fig.add_subplot(rows, cols, row * cols + col * 2 + 2)

        ax1.imshow(img)
        ax2.imshow(img)
        ax2.scatter(fixation.x, fixation.y, c='red', s=10)

        ax1.axis('off')
        ax2.axis('off')

        i += 1
        if i >= rows * cols // 2:
            break

    # plt.subplots_adjust(wspace=0, hspace=0) 
    plt.subplots_adjust(bottom=0.3, top=0.7, hspace=0, wspace = 0)
    plt.show()


create_image_grid(mit_stimuli, mit_fixations)

Cat Train set visualization !

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

def load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img = Image.open(os.path.join(folder, filename))
            if img is not None:
                images[filename.split('.')[0]] = img
    return images

def create_image_grid(original_images, saliency_maps, rows=18, cols=6):
    assert len(original_images) == len(saliency_maps)
    
    # Calculate the size of each subplot to fit the grid perfectly
    subplot_width = 1.0 / cols
    subplot_height = 1.0 / rows

    fig = plt.figure(figsize=(20, 85))

    i = 0
    for key, original_img in original_images.items():
        saliency_img = saliency_maps.get(key + '_SaliencyMap')
        if not saliency_img:
            continue

        row, col = divmod(i, cols // 2)

        ax1 = fig.add_subplot(rows, cols, row * cols + col * 2 + 1)
        ax2 = fig.add_subplot(rows, cols, row * cols + col * 2 + 2)

        ax1.imshow(original_img)
        ax2.imshow(saliency_img)

        ax1.axis('off')
        ax2.axis('off')

        i += 1
        if i >= rows * cols // 2:
            break

    # plt.subplots_adjust(wspace=0, hspace=0) 
    plt.subplots_adjust(bottom=0.0, top=0.0, hspace=0, wspace = 0)
    plt.show()

original_images = load_images_from_folder('../../datasets/cat2000/testSet/Stimuli/Art')
saliency_maps = load_images_from_folder('../../datasets/cat2000/testSet/Stimuli/Art/Output/')

create_image_grid(original_images, saliency_maps)


Atttain the spatial information, brightness, colorfulness and the contrast of the dataeset!

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import color, filters

def is_grayscale(img):
    """Check if the image is grayscale."""
    if len(img.shape) < 3: return True
    if img.shape[2] == 1: return True
    return np.all(img[:, :, 0] == img[:, :, 1]) and np.all(img[:, :, 1] == img[:, :, 2])

def contrast(img):
    """Contrast for grayscale or color images."""
    if is_grayscale(img):
        std_dev = np.std(img)
    else:
        gray_img = color.rgb2gray(img)
        std_dev = np.std(gray_img)
    return std_dev

def colorfulness(img):
    """Colorfulness metric, return 0 for grayscale images."""
    if is_grayscale(img):
        return 0
    R, G, B = img[:, :, 0], img[:, :, 1], img[:, :, 2]
    rg = R - G
    yb = 0.5 * (R + G) - B
    std_rg, std_yb = np.std(rg), np.std(yb)
    mean_rg, mean_yb = np.mean(rg), np.mean(yb)
    return np.sqrt(std_rg**2 + std_yb**2 + 0.3 * (mean_rg**2 + mean_yb**2))

def spatial_info(img):
    """Spatial information for grayscale or color images."""
    if is_grayscale(img):
        Gmag = filters.sobel(img)
    else:
        gray_img = color.rgb2gray(img)
        Gmag = filters.sobel(gray_img)
    Gdir = np.arctan2(np.gradient(Gmag)[0], np.gradient(Gmag)[1])
    return 0.5 * (np.std(Gmag) + np.std(Gdir))

def brightness(img):
    """Brightness for grayscale or color images."""
    if is_grayscale(img):
        return img
    R, G, B = img[:, :, 0], img[:, :, 1], img[:, :, 2]
    return 0.299 * R + 0.587 * G + 0.114 * B


def process_images(image_folder, dataset_name):
    # print("dataset name is", dataset_name)
    brightness_vals, contrast_vals, colorfulness_vals, spatial_vals = [], [], [], []
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            print(os.path.join(image_folder, filename))
            img = Image.open(os.path.join(image_folder, filename))
            img_np = np.array(img)
            if img_np.shape[-1] == 4 and len(img_np.shape) == 3:
                img_np = img_np[...,:3]
            brightness_vals.append(np.mean(brightness(img_np)))
            contrast_vals.append(contrast(img_np))
            colorfulness_vals.append(colorfulness(img_np))
            spatial_vals.append(spatial_info(img_np))

    return {
    'name': dataset_name,
    'brightness': brightness_vals,
    'contrast': contrast_vals,
    'colorfulness': colorfulness_vals,
    'spatial': spatial_vals
    }

def plot_distributions(datasets, metric):
    for dataset in datasets:
        sns.kdeplot(dataset[metric], label=dataset['name'])
    plt.title(f"{metric.capitalize()} Distribution")
    plt.legend()
    plt.show()

# Modified plot_distributions function
def plot_distributions(datasets):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))  # 1 row, 4 columns of subplots
    metrics = ['brightness', 'contrast', 'colorfulness', 'spatial']
    for i, metric in enumerate(metrics):
        ax = axes[i]
        for dataset in datasets:
            sns.kdeplot(dataset[metric], label=dataset['name'], ax=ax)
        ax.set_title(f"{metric.capitalize()} Distribution")
        ax.legend()
        ax.set_xlabel(metric.capitalize())
    plt.tight_layout()
    plt.show()
    
# Path to your image dataset
image_folder = '/home/liilu/Desktop/COURSE/ImageProcessing/Assignment2/BASELINE/datasets/DUT-OMRON/Stimuli'

# List of datasets
datasets_info = [
    {
        'path': '../../datasets/cat2000/testSet/Stimuli/Art',
        'name': 'CAT2000'
    },
    {
        'path': '../../datasets/DUT-OMRON/Stimuli',
        'name': 'DUT-OMRON'
    },
    {
        'path': '../../datasets/FIGRIM/Stimuli/Fillers/highway',
        'name': 'FIGRIM'
    },
    {
        'path': '../../datasets/MIT300/stimuli',
        'name': 'MIT300'
    },
    {
        'path': '../../datasets/NUSEF_public/Stimuli',
        'name': 'NUSEF-public'
    },
    {
        'path': '../../datasets/OSIE/Stimuli',
        'name': 'OSIE'
    },
    {
        'path': '../../datasets/PASCAL-S/Stimuli',
        'name': 'PASCAL-S'
    },    
    {
        'path': '../../datasets/SALICON/stimuli/train',
        'name': 'SALICON'
    },    
    {
        'path': '../../datasets/sjtuvis/stimuli',
        'name': 'sjtuvis'
    },
    {
        'path': '../../datasets/toronto/stimuli',
        'name': 'toronto'
    },
]

processed_datasets = [process_images(info['path'], info['name']) for info in datasets_info]

# Plotting distributions for each metric
for metric in ['brightness', 'contrast', 'colorfulness', 'spatial']:
    plot_distributions(processed_datasets)

Pysaliency
==========

Saliency Map Models
----------------------

`pysaliency` comes with a variety of features to evaluate saliency map models. This notebooks demonstrates these features.

First we load the MIT1003 dataset:

In [ ]:
import sys
import pysaliency
from pysaliency.external_datasets.sjtuvis import TextDescriptor

data_location = "../../datasets/test"
text_descriptor = TextDescriptor('../../datasets/test/original_sjtuvis_dataset/text.xlsx')
print(text_descriptor.get_description('000000020777_2.png')) 
mit_stimuli, mit_fixations = pysaliency.external_datasets.get_sjtu_vis("../../datasets/test/original_sjtuvis_dataset", location=data_location, text_descriptor=text_descriptor)

As some evaluation methods can take quite a long time to run, we prepare a smaller dataset consisting of only the first 10 stimuli:

We will use the saliency model *AIM* by Bruce and Tsotos

In [ ]:
data_location = "../../datasets"
mit_stimuli, mit_fixations = pysaliency.external_datasets.get_mit1003(location=data_location)
index = 10
plt.imshow(mit_stimuli.stimuli[index])
f = mit_fixations[mit_fixations.n == index]
plt.scatter(f.x, f.y, color='r')
_ = plt.axis('off')
cutoff = 20

model = pysaliency.external_models.A>IM(location="../../models/", 	cache_location=os.path.join('../../models/model_caches', 'AIM'), caching=True)
text_descriptor = TextDescriptor('../../datasets/test/original_sjtuvis_dataset/text.xlsx')
mit_stimuli, mit_fixations = pysaliency.external_datasets.get_sjtu_vis(original_dataset_path="../../datasets/test/original_sjtuvis_dataset", location='../../datasets', text_descriptor=text_descriptor)
cutoff = 10
short_stimuli = pysaliency.FileStimuli(filenames=mit_stimuli.filenames[:cutoff])
short_fixations = mit_fixations[mit_fixations.n < cutoff]
auc_uniform = model.AUC(short_stimuli, short_fixations, nonfixations='uniform', verbose=True)
auc_shuffled = model.AUC(short_stimuli, short_fixations, nonfixations='shuffled', verbose=True)
auc_identical_nonfixations = model.AUC(short_stimuli, short_fixations, nonfixations=short_fixations, verbose=True)
kl_uniform = model.fixation_based_KL_divergence(short_stimuli, short_fixations, nonfixations='uniform')
kl_shuffled = model.fixation_based_KL_divergence(short_stimuli, short_fixations, nonfixations='shuffled')
kl_identical = model.fixation_based_KL_divergence(short_stimuli, short_fixations, nonfixations=short_fixations)
nss = model.NSS(short_stimuli, short_fixations)
gold_standard = pysaliency.FixationMap(short_stimuli, short_fixations, kernel_size=30)
image_based_kl = model.image_based_kl_divergence(short_stimuli, gold_standard)
cc = model.CC(short_stimuli, gold_standard)
ssim = model.SIM(short_stimuli, gold_standard)
print("AUC Uniform: ", auc_uniform, "AUC Shuffled: ", auc_shuffled, "AUC Identical: ", auc_identical_nonfixations, "KL Uniform: ", kl_uniform, "KL Shuffled: ", kl_shuffled, "KL Identical: ", kl_identical, "NSS: ", nss, "Image Based KL: ", image_based_kl, "CC: ", cc, "SSIM: ", ssim)

from pysaliency.external_datasets.sjtuvis import TextDescriptor
text_descriptor = TextDescriptor('../../datasets/test/original_sjtuvis_dataset/text.xlsx')
data_location = "../../datasets/test"
original_dataset_path = "../../datasets/test/original_sjtuvis_dataset"
mit_stimuli, mit_fixations = pysaliency.external_datasets.get_sjtu_vis(original_dataset_path=original_dataset_path, location=data_location, text_descriptor = text_descriptor)
short_stimuli = pysaliency.FileStimuli(filenames=mit_stimuli.filenames[:cutoff])
short_fixations = mit_fixations[mit_fixations.n < cutoff]


Lavis text guided saliency model. Loaded from the lavis module. 😎 😎 😎

In [ ]:
import torch
from PIL import Image
from lavis.models import load_model_and_preprocess
from lavis.processors import load_processor
from matplotlib import pyplot as plt
from lavis.common.gradcam import getAttMap
from lavis.models.blip_models.blip_image_text_matching import compute_gradcam
import numpy as np
from pysaliency.datasets import Stimulus, Fixations,  StimuliStimulus
import pysaliency

def handle_stimulus(stimulus):
    """
    Make sure that a stimulus is a `Stimulus`-object
    """
    if not isinstance(stimulus, Stimulus):
        stimulus = Stimulus(stimulus)
    return stimulus

import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import face
from scipy.ndimage import zoom
from scipy.special import logsumexp
import torch

import deepgaze_pytorch    
from typing import Optional, Tuple, List, Dict, Any, Union, Sequence
import pysaliency
DEVICE = 'cuda'

class DeepGazeSaliencyModel(pysaliency.SaliencyMapModel):
    
    def __init__(self, block_num=7, dst_w=720, version = 1):
        super().__init__()
        if version == 1:
            self.model = deepgaze_pytorch.DeepGazeI(pretrained=True).to(DEVICE)
            self.use_hist_scan_path = False
        elif version == 2:
            self.model = deepgaze_pytorch.DeepGazeIIE(pretrained=True).to(DEVICE)
            self.use_hist_scan_path = False
        elif version == 3:
            self.model = deepgaze_pytorch.DeepGazeIII(pretrained=True).to(DEVICE)
            self.use_hist_scan_path = True
        self.block_num = block_num
        self.dst_w = dst_w
        
    def _saliency_map(self, stimulus, text_description:Optional[str]=None):
        image = stimulus
        # print("The input image....")
        # print(image.shape, image.dtype, np.min(image), np.max(image))
        fixation_history_x = np.array([1024//2, 300, 500, 200, 200, 700])
        fixation_history_y = np.array([768//2, 300, 100, 300, 100, 500])

        centerbias_template = np.load('../../pretrained_weights/deepgaze/centerbias_mit1003.npy')
        centerbias = zoom(centerbias_template, (image.shape[0]/centerbias_template.shape[0], image.shape[1]/centerbias_template.shape[1]), order=0, mode='nearest')
        centerbias -= logsumexp(centerbias)

        image_tensor = torch.tensor([image.transpose(2, 0, 1)]).to(DEVICE)
        centerbias_tensor = torch.tensor([centerbias]).to(DEVICE)

        if self.use_hist_scan_path:
            x_hist_tensor = torch.tensor([fixation_history_x[self.model.included_fixations]]).to(DEVICE)
            y_hist_tensor = torch.tensor([fixation_history_y[self.model.included_fixations]]).to(DEVICE)
            log_density_prediction = self.model(image_tensor, centerbias_tensor, x_hist_tensor, y_hist_tensor)
        else:
            log_density_prediction = self.model(image_tensor, centerbias_tensor)

        if log_density_prediction.dim() == 2:
            pass
        elif log_density_prediction.dim() == 4:
            log_density_prediction = log_density_prediction[0, 0]
        elif log_density_prediction.dim() == 3:
            print(log_density_prediction.shape)
            assert log_density_prediction.shape[0] == 1
            log_density_prediction = log_density_prediction[0]
        assert log_density_prediction.dim() == 2, "log_density_prediction should be a 3D tensor, but is {}".format(log_density_prediction.shape)
        return log_density_prediction.detach().cpu().numpy()


class LavisTextGuidedSaliencyModel(pysaliency.SaliencyMapModel):
    
    def __init__(self, debug_vis_flag = False, cache_location=None, caching=True, memory_cache_size=None):
        super().__init__(cache_location, caching, memory_cache_size)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model, self.vis_processors, self.text_processors = load_model_and_preprocess("blip_image_text_matching", "large", device=self.device, is_eval=True)
        self.debug_vis_flag = debug_vis_flag
        self.deepgaze_model = DeepGazeSaliencyModel(version=3)
        
    def saliency_map(self, stimulus):
        """
        Get saliency map for given stimulus.

        To overwrite this function, overwrite `_saliency_map` as otherwise
        the caching mechanism is disabled.
        """
        if isinstance(stimulus, StimuliStimulus):
            filename = os.path.basename(stimulus.filename)
        elif isinstance(stimulus, Stimulus):
            filename = None
        elif isinstance(stimulus, np.ndarray):
            filename = None
        else:
            raise NotImplementedError
        stimulus = handle_stimulus(stimulus)
            
        try:
            text_description=text_descriptor.get_description(filename)
        except Exception as e:
            text_description=None
        # print(text_description, stimulus.filename)
        if text_description is None:
            return self.deepgaze_model.saliency_map(stimulus.stimulus_data)
        else:
            # print(text_description)
            pass
        if not self.caching:
            return self._saliency_map(stimulus.stimulus_data, text_description=text_description)
        stimulus_id = stimulus.stimulus_id
        if not stimulus_id in self._cache:
            temp = stimulus.stimulus_data
            self._cache[stimulus_id] = self._saliency_map(temp, text_description=text_description)
        return self._cache[stimulus_id]
    
    def _saliency_map(self, stimulus, text_description=None):
        
        # print(f"****************      {text_description}      *********************")
        
        return self.evaluate_saliency_map(stimulus[...,:3], "large", text_description)
    
    def evaluate_saliency_map(self, raw_image, model_path, caption, block_num=7, dst_w=720):
        # print(raw_image.shape, raw_image.dtype, np.max(raw_image), np.min(raw_image))
        raw_image = Image.fromarray(np.uint8(raw_image)).convert('RGB')
        norm_img = np.float32(raw_image) / 255

        # Preprocess image and text inputs
        img = self.vis_processors["eval"](raw_image).unsqueeze(0).to(self.device)
        txt = self.text_processors["eval"](caption)

        # Compute GradCam
        txt_tokens = self.model.tokenizer(txt, return_tensors="pt").to(self.device)
        gradcam, _ = compute_gradcam(self.model, img, txt, txt_tokens, block_num=block_num)

        # Average GradCam for the full image
        avg_gradcam = getAttMap(norm_img, gradcam[0][1], blur=True, overlap=False)
        
        if self.debug_vis_flag:
            fig, ax = plt.subplots(1, 1, figsize=(10, 10))
            ax.imshow(avg_gradcam)
            ax.set_yticks([])
            ax.set_xticks([])

        overall_gradcam_wrt_tokens = []
        
        num_image = len(txt_tokens.input_ids[0]) - 2 + 1
        if self.debug_vis_flag:
            fig, ax = plt.subplots(num_image, 1, figsize=(15, 5 * num_image))

        gradcam_iter = iter(gradcam[0][2:-1])
        token_id_iter = iter(txt_tokens.input_ids[0][1:-1])

        for i, (gradcam, token_id) in enumerate(zip(gradcam_iter, token_id_iter)):
            word = self.model.tokenizer.decode([token_id])
            gradcam_image = getAttMap(norm_img, gradcam, blur=True, overlap=False)
            overall_gradcam_wrt_tokens.append(gradcam_image)
            if self.debug_vis_flag:
                ax[i].imshow(gradcam_image)
                ax[i].set_yticks([])
                ax[i].set_xticks([])
                ax[i].set_xlabel(word)

        overall_gradcam_wrt_tokens = np.stack(overall_gradcam_wrt_tokens, axis =0)
        avg_gradcam_wrt_tokens = np.mean(overall_gradcam_wrt_tokens, axis=0)
        
        if self.debug_vis_flag:
            ax[num_image-1].imshow(avg_gradcam_wrt_tokens)
            ax[num_image-1].set_yticks([])
            ax[num_image-1].set_xticks([])
            ax[num_image-1].set_xlabel(caption)
            
            plt.show()
            
        print(avg_gradcam.shape, avg_gradcam.dtype, np.max(avg_gradcam), np.min(avg_gradcam))
        print(avg_gradcam_wrt_tokens.shape, avg_gradcam_wrt_tokens.dtype, np.max(avg_gradcam_wrt_tokens), np.min(avg_gradcam_wrt_tokens))
        return avg_gradcam_wrt_tokens


# model: pysaliency.SaliencyMapModel = LavisTextGuidedSaliencyModel(debug_vis_flag=True)
# smap = model.saliency_map(mit_stimuli[10])

2023-12-13 09:31:47.082850: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 09:31:47.203233: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


To implement you own saliency map model, inherit from `pysaliency.SaliencyMapModel` and implement the `_saliency_map` method.

In [ ]:
import os
model_zoos = [  "SUN", "IttiKoch", "Judd", "CovSal","AIM", "ours", "deepgazeIIE", "deepgazeI", "deepgazeIII"]
import pysaliency
from pysaliency.external_models import AIM, SUN, GBVSIttiKoch, Judd, IttiKoch, CovSal
from pysaliency.external_datasets.sjtuvis import TextDescriptor
import numpy as np
import matplotlib.pyplot as plt
from pysaliency.saliency_map_conversion import optimize_for_information_gain

from pysaliency.external_datasets import get_mit1003, get_sjtu_vis, get_cat2000_train, get_FIGRIM, get_mit300, get_mit1003, get_mit1003_onesize, get_SALICON, get_toronto, get_DUT_OMRON, get_OSIE, get_PASCAL_S, get_NUSEF_public

DATASET_MAPPINGS = {
	"mit1003": get_mit1003,
 	"mit1003_onesize": get_mit1003_onesize,
 	"sjtuvis": get_sjtu_vis,
	# "cat2000_train": get_cat2000_train,
	"figrim": get_FIGRIM,
	"salicon_eval": get_SALICON,
	"toronto": get_toronto,
 	"DUT_OMRON": get_DUT_OMRON,
	"OSIE": get_OSIE,
	"PASCAL_S": get_PASCAL_S,
	"NUSEF_public": get_NUSEF_public
}

import pandas as pd
columns = ['Model', 'Dataset', 'AUC_shuffled', 'AUC_uniform', 'KL_uniform', 'KL_shuffled', 'KL_identical_nonfixations', 'Image_based_KL_divergence']
results_df = pd.DataFrame(columns=columns)


for model_name in model_zoos[:5]:
	print(f"------------------------    Model: {model_name} -----------------------------------")
	model: pysaliency.SaliencyMapModel
	model_location = "../../models/"
	if model_name == "deepgazeI":
		model: pysaliency.SaliencyMapModel = DeepGazeSaliencyModel(version=1)
	elif model_name == "deepgazeIIE":
		model: pysaliency.SaliencyMapModel = DeepGazeSaliencyModel(version=2)
	elif model_name == "deepgazeIII":
		model: pysaliency.SaliencyMapModel = DeepGazeSaliencyModel(version=3)
	elif model_name == "ours":
		model: pysaliency.SaliencyMapModel = LavisTextGuidedSaliencyModel()
	elif model_name == "GoldStandard":
		raise NotImplementedError
	else:
		model: pysaliency.SaliencyMapModel = eval(model_name)(location = model_location, cache_location=os.path.join('../../datasets/model_caches/', model_name), caching=True)
	
	
	
	for dataset_key, dataset_func in DATASET_MAPPINGS.items():
		dataset_location = "../../datasets/"
		print(f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~{dataset_key}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
		if dataset_key == "sjtuvis":
			text_descriptor = TextDescriptor('../../datasets/test/original_sjtuvis_dataset/text.xlsx')
			mit_stimuli, mit_fixations = dataset_func(original_dataset_path="../../datasets/test/original_sjtuvis_dataset", location=dataset_location, text_descriptor=text_descriptor)
		elif dataset_key == "cat2000_train":
			mit_stimuli, mit_fixations = dataset_func(location=dataset_location, version = "1.1")
		elif dataset_key == "salicon_train":
			stimuli_train, stimuli_val, stimuli_test, fixations_train, fixations_val = dataset_func(location=dataset_location)
			mit_stimuli, mit_fixations = stimuli_train, fixations_train
		elif dataset_key == "salicon_eval":
			stimuli_train, stimuli_val, stimuli_test, fixations_train, fixations_val = dataset_func(location=dataset_location)
			mit_stimuli, mit_fixations = stimuli_val, fixations_val
		else:
			mit_stimuli, mit_fixations = dataset_func(location=dataset_location)
			
		cutoff = 10
		short_stimuli = pysaliency.FileStimuli(filenames=mit_stimuli.filenames[:cutoff])
		short_fixations = mit_fixations[mit_fixations.n < cutoff]
		
		# try:
		auc_uniform = model.AUC(short_stimuli, short_fixations, nonfixations='uniform', verbose=True)
		auc_shuffled = model.AUC(short_stimuli, short_fixations, nonfixations='shuffled', verbose=True)
		auc_identical_nonfixations = model.AUC(short_stimuli, short_fixations, nonfixations=short_fixations, verbose=True)
		kl_uniform = model.fixation_based_KL_divergence(short_stimuli, short_fixations, nonfixations='uniform')
		# kl_shuffled = model.fixation_based_KL_divergence(short_stimuli, short_fixations, nonfixations='shuffled')
		kl_identical = model.fixation_based_KL_divergence(short_stimuli, short_fixations, nonfixations=short_fixations)
		nss = model.NSS(short_stimuli, short_fixations)
		gold_standard = pysaliency.FixationMap(short_stimuli, short_fixations, kernel_size=30)
		image_based_kl = model.image_based_kl_divergence(short_stimuli, gold_standard)
		cc = model.CC(short_stimuli, gold_standard)
		ssim = model.SIM(short_stimuli, gold_standard)
		# ig = optimize_for_information_gain(model, short_stimuli, short_fixations)
		# except Exception as e:
      
		# 	auc_uniform, auc_shuffled, auc_identical_nonfixations, kl_uniform, kl_identical, nss, image_based_kl, cc, ssim = np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

		result = {
			'Model': model_name,
			'Dataset': dataset_key,
			'AUC_shuffled': auc_shuffled,
			'AUC_uniform': auc_uniform,
			'NSS': nss,
			'KL_uniform': kl_uniform,	
			# 'KL_shuffled': kl_shuffled,
			'KL_identical_nonfixations': kl_identical,
			'Image_based_KL_divergence': image_based_kl,
			"cc": cc,
   			"ssim": ssim
		}
		results_df = pd.concat([results_df, pd.DataFrame([result])], ignore_index=True)
		
  

------------------------    Model: ours -----------------------------------


2023-12-13 09:31:59.048 | INFO     | lavis.models:load_model_and_preprocess:152 - Loading the preprocessors from the default config file...
2023-12-13 09:31:59.050 | INFO     | lavis.models:load_model_and_preprocess:153 - args:{'model': {'arch': 'blip_image_text_matching', 'load_finetuned': True, 'finetuned': 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_retrieval_coco.pth', 'vit_type': 'large', 'vit_grad_ckpt': False, 'vit_ckpt_layer': 0, 'image_size': 384, 'med_config_path': 'configs/models/med_large_config.json', 'embed_dim': 256}, 'preprocess': {'vis_processor': {'eval': {'name': 'blip_image_eval', 'image_size': 384}}, 'text_processor': {'eval': {'name': 'blip_caption'}}}}
2023-12-13 09:31:59.050 | INFO     | lavis.models:load_preprocess:96 - Visual pretrained model: {'eval': {'name': 'blip_image_eval', 'image_size': 384}} | Text pretrained model: {'eval': {'name': 'blip_caption'}}
Using cache found in /mnt/homes/minghao/.cache/torch/hub/pytor

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~mit1003~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_1920060/1814523666.py:59: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  image_tensor = torch.tensor([image.transpose(2, 0, 1)]).to(DEVICE)
100%|██████████| 10/10 [00:00<00:00, 413.60it/s]


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~mit1003_onesize~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


...

/mnt/homes/minghao/anaconda3/envs/lavis/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/homes/minghao/anaconda3/envs/lavis/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loaded pretrained weights for efficientnet-b5


Using cache found in /mnt/homes/minghao/.cache/torch/hub/pytorch_vision_v0.6.0
/mnt/homes/minghao/anaconda3/envs/lavis/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /mnt/homes/minghao/.cache/torch/hub/pytorch_vision_v0.6.0
/mnt/homes/minghao/anaconda3/envs/lavis/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most u

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~mit1003~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


 60%|██████    | 6/10 [00:06<00:04,  1.11s/it]

In [4]:
# Display the DataFrame
# results_df.to_csv("results.csv")
print(results_df)
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x='Model', y='AUC_shuffled', hue='Dataset')
plt.title('AUC Shuffled by Model and Dataset')
plt.xticks(rotation=45)
plt.show()

NameError: name 'results_df' is not defined